<div class='alert alert-info' style='text-align: center'><h1>Brain Tumor Object Detection</h1>
- yet another MR processing notebook -</div>

#### In this notebook, we'll use YOLOv5 to try and detect tumors in brain MRs.
#### There are three models (pytorch weights), one for each plane, that work on the T1wCE series only.
#### They were trained on a small sample (~400 images each) of the RSNA dataset I hand labeled using makesense.ai.
#### The image plane is checked and the appropriate weights are used in detection/classification.
#### They perform object detection and classification (positive/negative) simulataneously.

##### Model Dataset -> https://www.kaggle.com/davidbroberts/brain-tumor-yolo-od
##### Training Notebook -> https://www.kaggle.com/davidbroberts/brain-tumor-yolo-od-train
##### Training Dataset -> https://www.kaggle.com/davidbroberts/brain-tumor-object-detection-datasets

- You can cross reference the classification accuracy by comparing to the study's MGMT value.
- I included classification for demonstration purposes only (it's not very accurate .. of course none of the models in this comp are).
- The OD models find a lot of false positives on the inferior axial images especially. Excluding non-diagnostic (all black, peripheral images etc) slices is important here.
- I created OD models for the other series types, but I did not include them in this demo. Let me know if you're interested in them.

In [ ]:
import os
import numpy as np
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
import cv2
import shutil
from shutil import copyfile
import path

In [ ]:
predict_dir = '/kaggle/working/predict'
label_dir = '/kaggle/working/runs/detect/exp/labels'
image_dir = '/kaggle/working/runs/detect/exp'

In [ ]:
# Make a dir to export jpgs to
if not os.path.exists(f'{predict_dir}'):
    os.mkdir(f'{predict_dir}')

In [ ]:
# Copy YOLO
if not os.path.exists('/kaggle/working/yolov5'):
    shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')

In [ ]:
# This function gives a basic plane from the ImageOrientationPatient tag. It doesn't account for obliqueness. But we don't need to care about it.
# Will return 'unknown' if the image isn't exactly orthogonal.
def get_image_plane(loc):
    row_x = round(loc[0])
    row_y = round(loc[1])
    row_z = round(loc[2])
    col_x = round(loc[3])
    col_y = round(loc[4])
    col_z = round(loc[5])
    if (row_x, row_y, col_x, col_y) == (1,0,0,0):
        return "coronal"
    if (row_x, row_y, col_x, col_y) == (0,1,0,0):
        return "sagittal"
    if (row_x, row_y, col_x, col_y) == (1,0,0,1):
        return "axial"
    return "Unknown"

In [ ]:
# Call yolo detect.py on an image
def detect(plane):
    !python yolov5/detect.py --source {predict_dir} --weights ../input/brain-tumor-yolo-od/{plane}_t1wce_2_class.pt --img 512 --exist-ok --save-txt

In [ ]:
# Delete all images and labels
def cleanup():
    if os.path.exists(label_dir):
        filelist = [ f for f in os.listdir(label_dir) if f.endswith(".txt") ]
        for f in filelist:
            os.remove(os.path.join(label_dir, f))

        filelist = [ f for f in os.listdir(image_dir) if f.endswith(".jpg") ]
        for f in filelist:
            os.remove(os.path.join(image_dir, f))
            
    filelist = [ f for f in os.listdir(predict_dir) if f.endswith(".jpg") ]
    for f in filelist:
        os.remove(os.path.join(predict_dir, f))  

In [ ]:
def detect_tumor(study, image_number):

    # Make sure there aren't files hanging around from the last run
    cleanup()

    # Load an image
    image = pydicom.dcmread(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{study}/T1wCE/Image-{image_number}.dcm')
    pixels = image.pixel_array

    # Crunch pixels down to 8 bit
    pixels = pixels - np.min(pixels)
    pixels = pixels / np.max(pixels)
    pixels = (pixels * 255).astype(np.uint8)

    # Get the plane
    plane = get_image_plane(image.ImageOrientationPatient)
    print("Plane:", plane)

    # Expor the image as a JPG
    filename = f'{predict_dir}/{study}_t1wce_{image_number}.jpg'
    cv2.imwrite(filename, pixels)
    
    # Run YOLO detect on the exported image
    detect(plane)
    
    # Get the YOLO image and label/BB coords .. if they exist
    image_name = f'{image_dir}/{study}_t1wce_{image_number}.jpg'
    if os.path.isfile(image_name):
        img = cv2.imread(image_name)

        label_name = f'{label_dir}/{study}_t1wce_{image_number}.txt'
        if os.path.isfile(label_name):
            label_file = open(label_name, "r")
            label_text = label_file.read()

            fig, axes = plt.subplots(nrows=1, ncols=2,sharex=False, sharey=False, figsize=(10, 10))
            ax = axes.ravel()
            ax[0].set_title('Original')
            ax[0].imshow(pixels, cmap='gray')
            ax[1].set_title('OD Detect')
            ax[1].imshow(img, cmap='gray')

            plt.tight_layout()
            plt.show()

            print("Label/BB coords:", label_text)
        else:
            plt.figure(figsize= (6,6))
            plt.title('No Tumor Detected')
            plt.imshow(pixels, cmap='gray');

In [ ]:
# Specify a study,series and image number and call the main function
study = '00216'
image_number = '98'

detect_tumor(study, image_number)

#### The bounding box coords can be used to extract 'patches' or to aid with segmentation.
- Let's try some other images

In [ ]:
study = '00006'
image_number = '90'
detect_tumor(study, image_number)

- Sometimes the OD will predict two tumors in one. Like above.

In [ ]:
study = '00019'
image_number = '50'
detect_tumor(study, image_number)

In [ ]:
study = '00008'
image_number = '80'
detect_tumor(study, image_number)

#### - This tool can be useful to find tumors if the images are sorted well and poor images are removed prior to prediction.

Some of my other MR processing notebooks:

- Determining MR image planes -> https://www.kaggle.com/davidbroberts/determining-mr-image-planes
- Determining MR Slice Orientation -> https://www.kaggle.com/davidbroberts/determining-mr-slice-orientation
- Determining DICOM image order -> https://www.kaggle.com/davidbroberts/determining-dicom-image-order
- Reference Lines on MR images -> https://www.kaggle.com/davidbroberts/mr-reference-lines
- Manual VOI LUT on MR images -> https://www.kaggle.com/davidbroberts/manual-voi-lut-on-mr-images
- Standardizing MR Images -> https://www.kaggle.com/davidbroberts/standardizing-mr-images
- Export DICOM Images by Plane -> https://www.kaggle.com/davidbroberts/export-dicom-series-by-plane/